Text Extraction

In [6]:
import cv2 
import os
import sys
import glob
from os import path
sys.path.append(os.path.abspath(os.path.join('../scripts')))
import pandas as pd
from text_extraction import Text_extraction

In [7]:
text_extract=Text_extraction()
assets = glob.glob('../data/Assets/*')

In [8]:
def createDetector():
    detector = cv2.ORB_create(nfeatures=2000)
    return detector

In [9]:
def getFeatures(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detector = createDetector()
    kps, descs = detector.detectAndCompute(gray, None)
    return kps, descs, img.shape[:2][::-1]

In [10]:
def detectFeatures(img, train_features):
    train_kps, train_descs, shape = train_features
    # get features from input image
    kps, descs, _ = getFeatures(img)
    # check if keypoints are extracted
    if not kps:
        return None
    # now we need to find matching keypoints in two sets of descriptors (from sample image, and from current image)
    # knnMatch uses k-nearest neighbors algorithm for that
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.knnMatch(train_descs, descs, k=2)
    good = []
    # apply ratio test to matches of each keypoint
    # idea is if train KP have a matching KP on image, it will be much closer than next closest non-matching KP,
    # otherwise, all KPs will be almost equally far
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append([m])
    # stop if we didn't find enough matching keypoints
    if len(good) < 0.1 * len(train_kps):
        return None
    # estimate a transformation matrix which maps keypoints from train image coordinates to sample image
    src_pts = np.float32([train_kps[m[0].queryIdx].pt for m in good
                          ]).reshape(-1, 1, 2)
    dst_pts = np.float32([kps[m[0].trainIdx].pt for m in good
                          ]).reshape(-1, 1, 2)
    m, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    if m is not None:
        # apply perspective transform to train image corners to get a bounding box coordinates on a sample image
        scene_points = cv2.perspectiveTransform(np.float32([(0, 0), (0, shape[0] - 1), (shape[1] - 1, shape[0] - 1), (shape[1] - 1, 0)]).reshape(-1, 1, 2), m)
        rect = cv2.minAreaRect(scene_points)
        # check resulting rect ratio knowing we have almost square train image
        if rect[1][1] > 0 and 0.8 < (rect[1][0] / rect[1][1]) < 1.2:
            return rect
    return None

In [11]:
# get train features
img = cv2.imread('../data/Assets/3bc12d2bdea947d4502e885b6d9aa442/logo.png')
train_features = features.getFeatures(img)
# detect features on test image
region = features.detectFeatures(frame, train_features)
if region is not None:
    # draw rotated bounding box
    box = cv2.boxPoints(region)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
# display the image
cv2.imshow("Preview", img)

NameError: name 'cv2' is not defined

In [15]:
from logo_extraction import  MatchingDetector
t_matching=MatchingDetector('img')
folder_list = glob.glob('../data/Assets/*')
logo_feature = []
for folder in folder_list:
    query_img = os.path.join(folder,'_preview.png')
    train_img = os.path.join(folder,'logo.png')
   
    if os.path.exists(query_img) and os.path.exists(train_img):
        location, bottom_right,top_left, res, img = t_matching.template_matching_image(train_img, query_img,method=cv2.TM_CCOEFF_NORMED)
        if res is not None:
            logo_feature.append([folder.split('/')[-1],location[0],location[1],bottom_right[0],bottom_right[1],top_left[0],top_left[1]])
        else:
            logo_feature.append([folder.split('/')[-1],0,0,0,0,0,0]) 
    else:
        logo_feature.append([folder.split('/')[-1],0,0,0,0,0,0]) 

NameError: name 'cv2' is not defined